<a href="https://colab.research.google.com/github/BorisLoveDev/agents-of-uniform-difficulty/blob/main/get_answers_table_and_fit10rcnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
import random
np.random.seed(42)
random.seed(42)

In [2]:
from google.colab import drive
# Подключение Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
# Загрузка и нормализация данных MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

num_classes = 10

# Добавление измерения канала к данным
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

# Применение one-hot encoding
y_test_ohe = to_categorical(y_test, num_classes)
y_train_ohe = to_categorical(y_train, num_classes)
y_train_ohe.shape

(60000, 10)

In [4]:
# Загрузка данных для уровня сложности 0
x_data_level_0 = np.load('/content/drive/MyDrive/all_difficulty_data_mnist/x_data_difficult_level_0.npy')
y_data_level_0 = np.load('/content/drive/MyDrive/all_difficulty_data_mnist/y_data_difficult_level_0.npy')

# Загрузка данных для уровня сложности 6
x_data_level_6 = np.load('/content/drive/MyDrive/all_difficulty_data_mnist/x_data_difficult_level_6.npy')
y_data_level_6 = np.load('/content/drive/MyDrive/all_difficulty_data_mnist/y_data_difficult_level_6.npy')


In [5]:
# Функция для деформации изображения, как определено ранее
def deform_image_optimized(image, A, B, M=28, NP=5):
    C = M / (NP + 1.0)
    XN, YN = np.zeros(M), np.zeros(M)
    DX, DY = np.linspace(0, M-1, M), np.linspace(0, M-1, M)

    for j in range(NP):
        TXN = (j + 0.5 - np.random.random()) * C
        TYN = (j + 0.5 - np.random.random()) * C
        TDX = (j + 0.5 - np.random.random()) * C
        TDY = (j + 0.5 - np.random.random()) * C
        AXN = B * (1.0 - 2.0 * np.random.random())
        AYN = B * (1.0 - 2.0 * np.random.random())
        ADX = A * (1.0 - 2.0 * np.random.random())
        ADY = A * (1.0 - 2.0 * np.random.random())
        PXN = (0.1 + 0.9 * np.random.random()) * C
        PYN = (0.1 + 0.9 * np.random.random()) * C
        PDX = (0.1 + 0.9 * np.random.random()) * C
        PDY = (0.1 + 0.9 * np.random.random()) * C

        DX += ADX * np.exp(-((DX - TDX) / PDX)**2)
        DY += ADY * np.exp(-((DY - TDY) / PDY)**2)
        XN += AXN * np.exp(-((DX - TXN) / PXN)**2)
        YN += AYN * np.exp(-((DY - TYN) / PYN)**2)

    deformed_image = np.zeros((M, M))
    for j in range(M):
        for i in range(M):
            x_index = int(DX[i] + XN[j])
            y_index = int(DY[j] + YN[i])
            if 0 <= x_index < M and 0 <= y_index < M:
                deformed_image[j, i] = image[y_index, x_index] if y_index < image.shape[0] and x_index < image.shape[1] else 0

    return deformed_image

In [6]:
deform_params = {
    "1": (0.2, 1.0),
    "2": (0.4, 3.0),
    "3": (1, 5.0),
    "4": (3.5, 9.5),
    "5": (1, 5.0),
}

Получение таблицы ответов для 10 акторов

In [21]:
from tensorflow.keras.layers import Conv2D, SeparableConv2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D, Dense, Dropout, Flatten, BatchNormalization, Activation, LeakyReLU, ELU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
import tensorflow as tf


In [22]:

def create_random_cnn_model(input_shape=(28, 28, 1), num_classes=10):
    model = Sequential()

    conv_layer = random.choice([Conv2D, SeparableConv2D])
    filters = random.randint(16, 64)
    kernel_size = random.choice([(3, 3), (5, 5)])
    activation = random.choice(['relu', 'leaky_relu', 'elu', 'selu', 'swish'])

    model.add(conv_layer(filters=filters, kernel_size=kernel_size, activation=activation, input_shape=input_shape))

    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    num_layers = random.randint(1, 3)
    for _ in range(num_layers):
        conv_layer = random.choice([Conv2D, SeparableConv2D])
        filters = random.randint(16, 128)
        kernel_size = random.choice([(3, 3), (5, 5)])
        activation = random.choice(['relu', 'leaky_relu', 'elu', 'selu', 'swish'])

        if activation == 'selu':
            model.add(conv_layer(filters=filters, kernel_size=kernel_size, activation='selu', padding='same'))
        else:
            model.add(conv_layer(filters=filters, kernel_size=kernel_size, padding='same'))
            if activation == 'leaky_relu':
                model.add(LeakyReLU(alpha=0.01))
            elif activation == 'elu':
                model.add(ELU(alpha=1.0))
            elif activation == 'swish':
                model.add(Activation(tf.nn.swish))
            else:  # relu
                model.add(Activation(activation))

        model.add(BatchNormalization())
        if random.random() < 0.5:
            model.add(MaxPooling2D(pool_size=(2, 2)))
        if random.random() < 0.5:
            model.add(Dropout(rate=random.uniform(0.2, 0.5)))

    model.add(GlobalAveragePooling2D())
    model.add(Dense(units=random.randint(64, 128), activation='relu'))
    if random.random() < 0.5:
        model.add(Dropout(rate=random.uniform(0.2, 0.5)))
    model.add(Dense(num_classes, activation='softmax'))

    return model

# Пример использования
model = create_random_cnn_model(input_shape=(28, 28, 1), num_classes=10)
model.compile(optimizer=Adam(), loss=CategoricalCrossentropy(), metrics=['accuracy'])


In [39]:
def split_dataset_with_overlap(data, labels, num_chunks, overlap=0.2):
    """
    Делит набор данных на `num_chunks` частей с перекрытием `overlap`.

    Parameters:
    data (np.ndarray): Набор данных для деления.
    labels (np.ndarray): Метки классов соответствующих данных.
    num_chunks (int): Количество частей, на которые будет разделен набор данных.
    overlap (float): Доля перекрытия между последовательными частями.

    Returns:
    list of tuples: Список кортежей, где каждый кортеж содержит часть данных и соответствующие метки.
    """
    chunk_size = len(data) // num_chunks
    overlap_size = int(chunk_size * overlap)

    chunks = []
    for i in range(num_chunks):
        start = i * chunk_size - i * overlap_size
        end = start + chunk_size + overlap_size

        # Зацикливание данных для обеспечения перекрытия
        if end > len(data):
            end -= len(data)
            chunk_data = np.concatenate((data[start:], data[:end]))
            chunk_labels = np.concatenate((labels[start:], labels[:end]))
        else:
            chunk_data = data[start:end]
            chunk_labels = labels[start:end]

        chunks.append((chunk_data, chunk_labels))

    return chunks

# Пример использования
# Предположим, у вас есть переменные x_train и y_train, содержащие тренировочные данные и метки соответственно
num_chunks = 10  # Например, разделим на 5 частей
chunks = split_dataset_with_overlap(x_train, y_train, num_chunks, overlap=0.2)

# Теперь вы можете обучить модель на каждом чанке данных
for i, (chunk_data, chunk_labels) in enumerate(chunks):
    print(f"Размер чанка №{i+1} = {chunk_data.shape}")
    # Обучите вашу модель так
    # Пример: model.fit(chunk_data, chunk_labels, epochs=10, batch_size=128)



Размер чанка№1 = (7200, 28, 28, 1)
Размер чанка№2 = (7200, 28, 28, 1)
Размер чанка№3 = (7200, 28, 28, 1)
Размер чанка№4 = (7200, 28, 28, 1)
Размер чанка№5 = (7200, 28, 28, 1)
Размер чанка№6 = (7200, 28, 28, 1)
Размер чанка№7 = (7200, 28, 28, 1)
Размер чанка№8 = (7200, 28, 28, 1)
Размер чанка№9 = (7200, 28, 28, 1)
Размер чанка№10 = (7200, 28, 28, 1)


In [40]:
# Генерируем 10 случайные модели
models = [create_random_cnn_model(input_shape=x_train.shape[1:], num_classes=num_classes) for _ in range(10)]

# Компилируем модели
for model in models:
    model.compile(optimizer=Adam(), loss=CategoricalCrossentropy(), metrics=['accuracy'])

# Сохранение моделей
save_dir = "/content/drive/MyDrive/all_difficulty_data_mnist/weights_100rcnn_normal_train_data"
os.makedirs(save_dir, exist_ok=True)

# Обучение моделей на соответствующих чанках
for i, (model, (chunk_data, chunk_labels)) in enumerate(zip(models, chunks)):
    print(f"Обучение модели {i+1} на чанке {i+1}")
    model.fit(chunk_data, chunk_labels, epochs=10, batch_size=128, validation_split=0.2, verbose=1)
    # Сохранение модели
    model_path = os.path.join(save_dir, f"model_{i+1}.h5")
    model.save(model_path)
    print(f"Модель {i+1} сохранена в {model_path}")

# Тестирование моделей на тестовом наборе данных
for i, model in enumerate(models):
    print(f"Тестирование модели {i+1}")
    loss, accuracy = model.evaluate(x_test, y_test_ohe, verbose=0)
    print(f"Точность модели {i+1} на тестовых данных: {accuracy * 100:.2f}%")

Обучение модели 1 на чанке 1
Epoch 1/10
45/45 [==============================] - 1s 17ms/step - loss: 0.0423 - accuracy: 0.9861 - val_loss: 0.0577 - val_accuracy: 0.9840
Epoch 2/10
45/45 [==============================] - 0s 10ms/step - loss: 0.0339 - accuracy: 0.9894 - val_loss: 0.0620 - val_accuracy: 0.9868
Epoch 3/10
45/45 [==============================] - 0s 10ms/step - loss: 0.0333 - accuracy: 0.9889 - val_loss: 0.0423 - val_accuracy: 0.9882
Epoch 4/10
45/45 [==============================] - 0s 10ms/step - loss: 0.0288 - accuracy: 0.9917 - val_loss: 0.0488 - val_accuracy: 0.9861
Epoch 5/10
45/45 [==============================] - 0s 10ms/step - loss: 0.0266 - accuracy: 0.9903 - val_loss: 0.0428 - val_accuracy: 0.9896
Epoch 6/10
45/45 [==============================] - 0s 10ms/step - loss: 0.0225 - accuracy: 0.9922 - val_loss: 0.0501 - val_accuracy: 0.9861
Epoch 7/10
45/45 [==============================] - 0s 10ms/step - loss: 0.0224 - accuracy: 0.9934 - val_loss: 0.0405 - val_a

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


45/45 [==============================] - 1s 13ms/step - loss: 0.1990 - accuracy: 0.9408 - val_loss: 0.0836 - val_accuracy: 0.9729
Epoch 2/10
45/45 [==============================] - 0s 11ms/step - loss: 0.1920 - accuracy: 0.9406 - val_loss: 0.1058 - val_accuracy: 0.9660
Epoch 3/10
45/45 [==============================] - 0s 11ms/step - loss: 0.1907 - accuracy: 0.9450 - val_loss: 0.1742 - val_accuracy: 0.9472
Epoch 4/10
45/45 [==============================] - 0s 11ms/step - loss: 0.1770 - accuracy: 0.9486 - val_loss: 0.1307 - val_accuracy: 0.9542
Epoch 5/10
45/45 [==============================] - 1s 11ms/step - loss: 0.1665 - accuracy: 0.9490 - val_loss: 0.1367 - val_accuracy: 0.9528
Epoch 6/10
45/45 [==============================] - 0s 11ms/step - loss: 0.1745 - accuracy: 0.9443 - val_loss: 0.1182 - val_accuracy: 0.9611
Epoch 7/10
45/45 [==============================] - 1s 11ms/step - loss: 0.1770 - accuracy: 0.9479 - val_loss: 0.1079 - val_accuracy: 0.9639
Epoch 8/10
45/45 [======

In [2]:
теперь давай применим к тестовым данным
def generate_deformed_data(x_data, deform_params):(еще не задана в гугл коллабе)
    # Создание словаря для хранения деформированных данных по уровням сложности
    deformed_data = {}
    for level, params in deform_params.items():
        A, B = params
        # Деформация всех тестовых изображений согласно текущим параметрам деформации
        deformed_images = np.array([deform_image_optimized(image.reshape(28, 28), A, B) for image in x_data]).reshape(-1, 28, 28, 1)
        deformed_data[level] = deformed_images
    return deformed_data

# Генерация деформированных тестовых данных для каждого уровня сложности
deformed_test_data = generate_deformed_data(x_test, deform_params)
с параметрами(уже заданы в коде коллаба выше
deform_params = {
    "1": (0.2, 1.0),
    "2": (0.4, 3.0),
    "3": (1, 5.0),
    "4": (3.5, 9.5),
    "5": (1, 5.0),
}
нужно поделить тестовый набор первые 1 тыс. экзмплряов из тестовых данных это деформация с параметрами 1, вторая тысяча с параметрами 2 и тд.
нужно создать тестовые деофрмированые данные всех видов 1 - 5 объеденить с
0 уровнем(возьми первую 1 тыс.) и 6 уровнем(возьми полсдение 1 тыс.)
6 уровню дай лейбл спец класс 10 не знаю
# Загрузка данных для уровня сложности 0
x_data_level_0 = np.load('/content/drive/MyDrive/all_difficulty_data_mnist/x_data_difficult_level_0.npy')
y_data_level_0 = np.load('/content/drive/MyDrive/all_difficulty_data_mnist/y_data_difficult_level_0.npy')

# Загрузка данных для уровня сложности 6
x_data_level_6 = np.load('/content/drive/MyDrive/all_difficulty_data_mnist/x_data_difficult_level_6.npy')
y_data_level_6 = np.load('/content/drive/MyDrive/all_difficulty_data_mnist/y_data_difficult_level_6.npy')

они уже дефорвмированы
потом прогнать каждую сеть и посчитать точность и таблицы cvs где будет номер сети
+1 если правильный ответ 0 если ответ не знаю -1 если неправлиьный

SyntaxError: invalid syntax (<ipython-input-2-8ac04ee40d02>, line 1)